In [3]:
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn import svm
from sklearn.linear_model import LogisticRegression
import torch
import matplotlib.pyplot as plt
from joblib import dump
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [4]:
X = pd.read_csv("../../processed_data.csv")
print(len(X[X.click_bool == 1.0]))
#X_train = X.head(2000000)
X_train = X.head(100000)
#X_val = X.tail(397656)
X_val = X.tail(50000)
len(X_val)

106079


50000

In [5]:
#Validation set

# Remove the label from the dataset and create Y vector
val = X_val.to_numpy()#[:100000,:]
print(val.shape)
# 8: click_bool, 9: booking_bool
exclude = [8,9, 10]
X_val = np.delete(val, exclude, axis=1)


(50000, 11)


In [6]:
Y_val = val[:,[8,9]]
Y_val_rel = np.zeros(Y_val.shape[0])
Y_val_rel[np.argwhere(Y_val[:,0]==1)[:,0]] = 2
Y_val_rel[np.argwhere(Y_val[:,1]==1)[:,0]] = 5
Y_val_rel[:100]

array([0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 5., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [7]:
len(X_train[X_train.click_bool == 1.0])

4393

In [8]:
X_zero = X_train[X_train.click_bool == 0.0].sample(7000, random_state=1)
X_one = X_train.loc[X_train.click_bool == 1.0]
train = pd.concat([X_zero, X_one], ignore_index=True, sort=False)
len(train)

11393

In [9]:
# Remove the label from the dataset and create Y vector
train = train.to_numpy()#[:100000,:]
print(train.shape)
# 8: click_bool, 9: booking_bool
exclude = [8,9, 10]
Y_train = train[:,8]
print(Y_train[:20])
X_train = np.delete(train, exclude, axis=1)
print(X_train.shape, Y_train.shape)

(11393, 11)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(11393, 8) (11393,)


In [10]:
(Y_train==0).sum()

7000

In [11]:
scaler = StandardScaler().fit(X_train)
dump(scaler, "scaler_svm_train.joblib")
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
print(X_train.shape)

(11393, 8)


In [ ]:
#Model = svm.SVC(decision_function_shape="ovr")
Model = svm.SVC(decision_function_shape="ovr", kernel="linear", verbose=True, probability=True)
Model.fit(X_train,Y_train)

#predictions = Model.predict(scaler.transform(X_val))
#print("Accuracy: ", np.mean(predictions==Y_val))
dump(Model, "svm.joblib_subset")

In [ ]:
"""
from sklearn.ensemble import RandomForestClassifier

Model = RandomForestClassifier(max_depth=2, random_state=0, class_weight="balanced")
Model.fit(X_train,Y_train)

predictions = Model.predict(scaler.transform(X_val))
print("Accuracy: ", np.mean(predictions==Y_val))


In [12]:
from sklearn.metrics import ndcg_score

In [ ]:
predictions = Model.predict_proba(scaler.transform(X_val))

In [ ]:
#print(Y_val_rel.shape)
#print(predictions[:,1].shape)

print(predictions[:,1].shape)

Y_val_rel = np.array([Y_val_rel])
prediction_rel = np.array([predictions[:,1]])
print(Y_val_rel.shape)
print(prediction_rel.shape)

In [ ]:
ndcg_score(Y_val_rel, prediction_rel)

In [13]:
import xgboost as xgb

In [14]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

xg_reg.fit(X_train,Y_train)

[19:08:07] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=5,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=10, n_jobs=12, num_parallel_tree=1,
             objective='reg:linear', random_state=0, reg_alpha=10, reg_lambda=1,
             scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [15]:
preds = xg_reg.predict(scaler.transform(X_val))
print(preds[:50])

[0.42143518 0.42143518 0.42143518 0.42143518 0.40326425 0.42143518
 0.40326425 0.40326425 0.42143518 0.40326425 0.42143518 0.40326425
 0.42143518 0.40326425 0.42143518 0.40326425 0.42143518 0.40326425
 0.40326425 0.42143518 0.40326425 0.42143518 0.42143518 0.42143518
 0.40326425 0.40326425 0.40326425 0.40326425 0.40326425 0.40326425
 0.42143518 0.42143518 0.42143518 0.40326425 0.40326425 0.42143518
 0.42143518 0.40326425 0.42143518 0.42143518 0.42143518 0.42143518
 0.40326425 0.40326425 0.40326425 0.40326425 0.40326425 0.40326425
 0.42318544 0.42318544]


In [19]:
ndcg_score(np.array([Y_val_rel]), np.array([preds]))

0.654291426044383